In [ ]:
from sqlalchemy import create_engine, text
import seaborn as sns
import pandas as pd

In [ ]:
user = "aabounak"
password = "mysecretpassword"
host = "localhost"
port = "5432"
database = "piscineds"

def connect_to_postgres():
  return create_engine(
    url=f"postgresql://{user}:{password}@{host}:{port}/{database}"
  )

In [ ]:
engine = connect_to_postgres()

In [ ]:
with engine.connect() as connection:
  result = connection.execute(text('select * from customers limit 10000000'))
  data = result.fetchall()

columns_names = result.keys()
df = pd.DataFrame(data, columns=columns_names)
df.head()

In [ ]:
sns.set_style("darkgrid")

In [ ]:
df['event_type'].value_counts().plot(kind='pie', autopct='%1.1f%%')